# Processamento dos Sinais de EEG - Dataset: Kolodziej

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
from scipy.io import loadmat
%matplotlib qt5

Importar os arquivos .mat em arrays (separando por frequências)

In [3]:
info_array_5hz = []
eeg_array_5hz = []
for i in range(1,6):
    info_array_5hz.append(loadmat('5Hz/S{}_5Hz.mat'.format(i)))
    eeg_array_5hz.append(info_array_5hz[i-1]['X'])
eeg_array_5hz = np.array(eeg_array_5hz)

O array eeg_5hz possui shape na configuração (subject, time, channel)

In [4]:
eeg_array_5hz.shape

(5, 7680, 16)

Utilizaremos para processamento o canal Oz (index 14)

In [5]:
Oz_array_5hz = []
for i in range(5):
    Oz_array_5hz.append(eeg_array_5hz[i,:,14])

Oz_array_5hz = np.array(Oz_array_5hz)
print(Oz_array_5hz.shape) # Configuração do shape: (subject, time)

(5, 7680)


In [6]:
# Definindo a função para gerar a DFT de uma sequência (visualização apenas no intervalo de interesse)

def plotDFT(input_signal, sampling_frequency):
    """Plota a DFT de input_signal. O intervalo de visualização é apenas de 0 a pi. 
    Input arguments: 
    input_signal: Sequência de entrada 
    sampling_frequency: frequência de amostragem
    """
    
    N = len(input_signal)
    T = 1. / sampling_frequency
    xf = np.linspace(0.,1./(2.*T), N/2)
    fft_input = np.fft.fft(input_signal)
    
    return plt.plot(xf, np.abs(fft_input[:N/2]))

In [56]:
#Plotando as Densidades espectrais para o Subject_1- Periodograma tradicional e Método de Welch

Oz_period = sig.periodogram(Oz_array_5hz[0], fs=256.,nfft=len(Oz_array_5hz[0]))
Oz_welch = sig.welch(Oz_array_5hz[0], fs=256., window='hann', nperseg=350, nfft=512)
#Oz_welch2 = sig.welch(Oz_array_5hz[0], fs=256., nperseg=350, noverlap=0.75,nfft=len(Oz_array_5hz[0]))

plt.figure()
plt.hold(True)
plt.plot(Oz_period[0],Oz_period[1],'b',label='Periodograma')
plt.plot(Oz_welch[0], Oz_welch[1],'r', label='Welch')
#plt.plot(Oz_welch2[0], Oz_welch2[1], 'g', label='Welch')

plt.legend()

ValueError: Unknown window type.

In [37]:
# Pré-processamento: Filtragem do sinal Oz_array_5hz[0]. Filtro: FIR 2-50 Hz ordem 50

fs = 256. 
f_norm = 2*np.pi / fs
Nyq_freq = fs / 2.
#filter_spec = [[wp1, wp2], [ws1, ws2]]
'''
wp = [2.*f_norm, 48.*f_norm]
ws = [1.*f_norm, 50.*f_norm]
'''
#W = np.array([[0.05, 48.],[0.01, 50.]]) / Nyq_freq
w = np.array([1, 50]) / Nyq_freq
gpass = 1.
gstop = 100.
n = 100
resp= sig.firwin(n, w, pass_zero=False, window='hann')

a, h = sig.freqz(resp)


plt.figure()
plt.title('Resposta em frequência do filtro passa banda', fontsize=16)
plt.xlabel('Frequência (Hz)', fontsize=16)
plt.ylabel('Amplitude (dB)', fontsize=16)
plt.plot((Nyq_freq/np.pi)*a,20*np.log10(np.abs(h)), 'b')
plt.grid()



In [49]:
# Periodograma do sinal filtrado
yfilt = sig.lfilter(resp, [1.], y)
yfilt_period = sig.periodogram(yfilt, fs=256., nfft=len(Oz_array_5hz[0]))

plt.plot(yfilt_period[0], yfilt_period[1])
plt.grid()


In [39]:
# Sinal filtrado versus sinal não filtrado em função do tempo
y = Oz_array_5hz[0]
t = np.arange(len(y)) / fs

plt.grid()
plt.title('Sinais filtrado e não filtrado', fontsize = 16)
plt.xlabel('tempo (s)', fontsize=16)
plt.ylabel('Amplitude (V)', fontsize=16)
plt.plot(t, y,'b',label='não filtrado')
plt.plot(t, yfilt, 'r', label='filtrado')
plt.legend()


In [41]:
# Avaliando a PSD do sinal filtrado utilizando o periodograma e o método de Welch

yfilt_welch = sig.welch(yfilt, fs=256., nperseg=500, noverlap=0.75, nfft=1024)
plt.grid()
plt.plot(yfilt_welch[0], yfilt_welch[1],'r')
plt.plot(yfilt_period[0], yfilt_period[1],'g')


In [22]:
max(yfilt_period[1])

4.8727660630379936e-11

In [28]:
f = ((Nyq_freq/np.pi)*a)

In [ ]:
print(f[])

In [35]:
np.where(20*np.log10(np.abs(h)) == -3)

(array([], dtype=int64),)